In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import cv2
import numpy as np
from matplotlib.lines import Line2D

**Divide the large tiff image into 256*256 blocks**

In [ ]:
def split_tif_pil(input_tif, output_folder, tile_size):
    os.makedirs(output_folder, exist_ok=True)

    Image.MAX_IMAGE_PIXELS = None
    img = Image.open(input_tif)
    width, height = img.size

    for i in range(0, width, tile_size):
        for j in range(0, height, tile_size):
            box = (i, j, min(i + tile_size, width), min(j + tile_size, height))
            tile = img.crop(box)

            output_path = os.path.join(output_folder, f"tile_{i}_{j}.tif")
            tile.save(output_path, format="TIFF")
            print(f"Saved {output_path}")


split_tif_pil(
    "../Data/skin_TXK6Z4X_D1_processed/tmap/wsi.tif", "output_tiles", tile_size=256
)

**Change cell maker image background color to transparent**

In [ ]:
A1_cells_layer_path = "../Data/skin_TXK6Z4X_A1_processed/cells_layer.png"
D1_cells_layer_path = "../Data/skin_TXK6Z4X_D1_processed/cells_layer.png"
image = cv2.imread(D1_cells_layer_path)

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

lower_red1 = np.array([0, 120, 70])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 120, 70])
upper_red2 = np.array([180, 255, 255])


mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Convert the image to RGBA
rgba = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
rgba[:, :, 3] = red_mask

cv2.imwrite("output.png", rgba)